In \[1\]:

    library(tidyverse)
    library(haven)
    library(dplyr)

    install.packages("stargazer")
    library(stargazer)

    ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

    ✔ ggplot2 3.3.6     ✔ purrr   0.3.4
    ✔ tibble  3.1.8     ✔ dplyr   1.0.9
    ✔ tidyr   1.2.0     ✔ stringr 1.4.0
    ✔ readr   2.1.2     ✔ forcats 0.5.1

    Warning message:
    “package ‘ggplot2’ was built under R version 4.1.3”
    Warning message:
    “package ‘tidyr’ was built under R version 4.1.2”
    Warning message:
    “package ‘readr’ was built under R version 4.1.2”
    Warning message:
    “package ‘dplyr’ was built under R version 4.1.3”
    ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
    ✖ dplyr::filter() masks stats::filter()
    ✖ dplyr::lag()    masks stats::lag()

    Warning message:
    “package ‘haven’ was built under R version 4.1.3”
    Updating HTML index of packages in '.Library'

    Making 'packages.html' ...
     done


    Please cite as: 


     Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

     R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 

In \[2\]:

    cchs <- read_dta("CCHS_Annual_2017_2018_curated_trimmed_25%.dta",
                     encoding = NULL,
                     c(hwtdgbmi,
                       hwtdgisw,
                       GEN_010,
                       GEN_015,
                       dhhgage,
                       CCC_095,
                       DHH_SEX,
                       paadvacv),
                     skip = 0,
                     n_max = Inf,
                     .name_repair = "unique") %>%
            drop_na() %>% 
            filter(hwtdgbmi != 999.96&999.99  & hwtdgisw != 6&9 & GEN_010 != 97&98&99 & GEN_015 != 7&8&9 & dhhgage != 01&02 & paadvacv != 6&9) %>%
            rename("BMI" = "hwtdgbmi",
                   "BMI_group" = "hwtdgisw",
                   "happy_score" = "GEN_010",
                   "mental_health" = "GEN_015",
                   "age" = "dhhgage",
                   "diabetes" = "CCC_095",
                   "gender" = "DHH_SEX",
                   "fitness" = "paadvacv") %>%
            mutate(BMI_group = as_factor(BMI_group),
                   age = as_factor(age),
                   diabetes = as_factor(diabetes),
                   gender = as_factor(gender),
                   fitness = as_factor(fitness)) %>%
            mutate(mental_health = case_when(
                mental_health == 1 ~ 5,
                mental_health == 2 ~ 4,
                mental_health == 3 ~ 3,
                mental_health == 4 ~ 2,
                mental_health == 5 ~ 1,
            )) %>% 
            mutate(fitness = case_when(
                fitness == "Physically active at / above recommended level from CPAG" ~ "High activity",
                fitness == "Physically active below recommended level from CPAG" ~ "Low activity",
                fitness == "No physical activity minutes reported" ~ "No activity")) %>%
            mutate(across(c('BMI'), round, 0))


    head(cchs)

| gender  | age                   | happy_score | mental_health | BMI     | BMI_group     | diabetes | fitness       |
|---------|-----------------------|-------------|---------------|---------|---------------|----------|---------------|
| \<fct\> | \<fct\>               | \<dbl+lbl\> | \<dbl\>       | \<dbl\> | \<fct\>       | \<fct\>  | \<chr\>       |
| Female  | Age between 45 and 49 | 10          | 5             | 20      | Normal weight | No       | Low activity  |
| Female  | Age between 50 and 54 | 6           | 3             | 23      | Normal weight | No       | No activity   |
| Female  | Age between 70 and 74 | 10          | 5             | 29      | Overweight    | No       | High activity |
| Female  | Age between 60 and 64 | 8           | 5             | 27      | Overweight    | No       | Low activity  |
| Male    | Age between 35 and 39 | 9           | 4             | 24      | Normal weight | No       | High activity |
| Male    | Age between 55 and 59 | 6           | 4             | 29      | Overweight    | No       | Low activity  |

A tibble: 6 × 8

In \[4\]:

    summary <- select(cchs, happy_score, BMI_group, diabetes) %>%
                group_by(BMI_group, diabetes) %>% 
                summarise(Mean_happyscore = mean(happy_score), 
                          Min_happyscore = min(happy_score),
                          Max_happyscore = max(happy_score), 
                          Sd_happyscore = sd(happy_score)) %>%
                ungroup()

    summary

    `summarise()` has grouped output by 'BMI_group'. You can override using the
    `.groups` argument.

| BMI_group                | diabetes | Mean_happyscore | Min_happyscore | Max_happyscore | Sd_happyscore |
|--------------------------|----------|-----------------|----------------|----------------|---------------|
| \<fct\>                  | \<fct\>  | \<dbl\>         | \<dbl+lbl\>    | \<dbl+lbl\>    | \<dbl\>       |
| Underweight              | Yes      | 7.083333        | 2              | 10             | 2.391589      |
| Underweight              | No       | 7.653061        | 0              | 10             | 1.965701      |
| Normal weight            | Yes      | 7.703371        | 0              | 10             | 1.975552      |
| Normal weight            | No       | 8.112955        | 0              | 10             | 1.621514      |
| Overweight               | Yes      | 7.702100        | 0              | 10             | 1.985948      |
| Overweight               | No       | 8.153255        | 0              | 10             | 1.588494      |
| Obese - Class I, II, III | Yes      | 7.491675        | 0              | 10             | 2.034308      |
| Obese - Class I, II, III | No       | 7.826238        | 0              | 10             | 1.796452      |

A tibble: 8 × 6

In \[6\]:

    simple <- lm(data = cchs, happy_score ~ BMI) #the effect of gender on wealth
    multi <- lm(data = cchs, happy_score ~ BMI + diabetes)
    interaction <- lm(data = cchs, happy_score ~ BMI*diabetes) #the interaction of BMI and diabetes on happpiness
    nonlinear <- lm(data = cchs, happy_score ~ BMI + I(log(BMI)) + diabetes)
    controls <- lm(data = cchs, happy_score ~ BMI+I(log(BMI)) +diabetes+ age+ gender + fitness)

    var_omit = c("(age)\\w+","(gender)\\w+")
    stargazer(multi, interaction, nonlinear,controls, 
              title="Comparison of Regression Results", omit = var_omit,
              align = TRUE, type="text", keep.stat = c("n","rsq", "adj.rsq", "ser","f"))

    Comparison of Regression Results
    ==============================================================================================================================
                                                                   Dependent variable:                                            
                        ----------------------------------------------------------------------------------------------------------
                                                                       happy_score                                                
                                   (1)                        (2)                       (3)                        (4)            
    ------------------------------------------------------------------------------------------------------------------------------
    BMI                         -0.019***                  -0.027***                 -0.210***                  -0.210***         
                                 (0.002)                    (0.007)                   (0.016)                    (0.017)          
                                                                                                                                  
    I(log(BMI))                                                                       5.358***                   5.403***         
                                                                                      (0.452)                    (0.463)          
                                                                                                                                  
    diabetesNo                   0.393***                    0.144                    0.394***                   0.367***         
                                 (0.038)                    (0.206)                   (0.038)                    (0.039)          
                                                                                                                                  
    BMI:diabetesNo                                           0.008                                                                
                                                            (0.007)                                                               
                                                                                                                                  
    fitnessLow activity                                                                                         -0.160***         
                                                                                                                 (0.027)          
                                                                                                                                  
    fitnessNo activity                                                                                          -0.335***         
                                                                                                                 (0.029)          
                                                                                                                                  
    Constant                     8.172***                  8.397***                  -4.225***                  -4.273***         
                                 (0.073)                    (0.197)                   (1.049)                    (1.074)          
                                                                                                                                  
    ------------------------------------------------------------------------------------------------------------------------------
    Observations                  23,818                    23,818                     23,818                     23,818          
    R2                            0.009                      0.009                     0.015                      0.024           
    Adjusted R2                   0.009                      0.009                     0.015                      0.023           
    Residual Std. Error     1.695 (df = 23815)        1.695 (df = 23814)         1.690 (df = 23814)         1.682 (df = 23798)    
    F Statistic         113.034*** (df = 2; 23815) 75.865*** (df = 3; 23814) 122.562*** (df = 3; 23814) 31.079*** (df = 19; 23798)
    ==============================================================================================================================
    Note:                                                                                              *p<0.1; **p<0.05; ***p<0.01

In \[7\]:

    df <- data.frame(BMI = cchs$BMI,
                     predicted1 = predict(multi),
                     predicted2 = predict(controls))

    ggplot(data = cchs, aes(x = BMI, y = happy_score)) +  
      geom_point() +
      geom_line(data = df, aes(x = BMI, y = predicted1), color = "blue") +
      geom_line(data = df, aes(x = BMI, y = predicted2), color = "green") +
      geom_smooth(method = "lm", se = FALSE, color = "red") +
      labs(x = "BMI", 
           y = "Satisfaction with life in general scale 1-10") +
      theme(text = element_text(size = 14))

    Don't know how to automatically pick scale for object of type haven_labelled/vctrs_vctr/double. Defaulting to continuous.

    `geom_smooth()` using formula 'y ~ x'

<img
src="..."
width="420" height="420" />

In \[8\]:

    library(lmtest)
    bptest_multi <- bptest(multi)
    bptest_interaction <- bptest(interaction)
    bptest_nonlinear <- bptest(nonlinear)
    bptest_controls <- bptest(controls)
    print(bptest_multi)
    print(bptest_interaction)
    print(bptest_nonlinear)
    print(bptest_controls)

    Warning message:
    “package ‘lmtest’ was built under R version 4.1.2”
    Loading required package: zoo

    Warning message:
    “package ‘zoo’ was built under R version 4.1.3”

    Attaching package: ‘zoo’


    The following objects are masked from ‘package:base’:

        as.Date, as.Date.numeric

        studentized Breusch-Pagan test

    data:  multi
    BP = 99.467, df = 2, p-value < 2.2e-16


        studentized Breusch-Pagan test

    data:  interaction
    BP = 100.62, df = 3, p-value < 2.2e-16


        studentized Breusch-Pagan test

    data:  nonlinear
    BP = 156.69, df = 3, p-value < 2.2e-16


        studentized Breusch-Pagan test

    data:  controls
    BP = 378.77, df = 19, p-value < 2.2e-16

In \[9\]:

    multi <- lm(data = cchs, happy_score ~ BMI+diabetes)
    robustness1 <- lm(data = cchs, mental_health ~ BMI + diabetes)

    var_omit = c("(age)\\w+","(gender)\\w+","(fitness)\\w+")
    stargazer(multi,robustness1, title="ALTERNATIVE SATISFACTION MEASUREMENT REGRESSION MODEL",
              align = TRUE, type="text", omit = var_omit, keep.stat = c("n","rsq", "adj.rsq", "ser","f"))

    ALTERNATIVE SATISFACTION MEASUREMENT REGRESSION MODEL
    =============================================================
                                         Dependent variable:     
                                     ----------------------------
                                      happy_score   mental_health
                                          (1)            (2)     
    -------------------------------------------------------------
    BMI                                -0.019***      -0.008***  
                                        (0.002)        (0.001)   
                                                                 
    diabetesNo                          0.393***      0.108***   
                                        (0.038)        (0.022)   
                                                                 
    Constant                            8.172***      4.002***   
                                        (0.073)        (0.041)   
                                                                 
    -------------------------------------------------------------
    Observations                         23,818        23,818    
    R2                                   0.009          0.003    
    Adjusted R2                          0.009          0.003    
    Residual Std. Error (df = 23815)     1.695          0.964    
    F Statistic (df = 2; 23815)        113.034***     39.899***  
    =============================================================
    Note:                             *p<0.1; **p<0.05; ***p<0.01